In [1]:
import pandas as pd
import re

# Cargar el archivo TXT inicial
file_path_initial = 'Interactions_intact_Fusobacterium_Nucleatum.txt'
data_initial = pd.read_csv(file_path_initial, delimiter='\t')

# Procesar las columnas de alias
data_initial['Alias(es) interactor A'] = data_initial['Alias(es) interactor A'].str.split('|')
data_initial['Alias(es) interactor B'] = data_initial['Alias(es) interactor B'].str.split('|')

# Función para extraer nombres de genes de un alias
def extract_gene_names(alias_list):
    gene_names = []
    if isinstance(alias_list, list):  # Asegurarse de que alias_list es una lista
        for alias in alias_list:
            # Buscar un nombre de gen que siga el patrón 'uniprotkb:GENE_NAME(gene name)'
            match = re.search(r'uniprotkb:([\w-]+)\(gene name\)', alias)
            if match:
                gene_names.append(match.group(1))
    return ','.join(gene_names)  # Convertir la lista de nombres a una cadena separada por comas

# Aplicar la función a las columnas de alias
data_initial['Interactor A Genes'] = data_initial['Alias(es) interactor A'].apply(extract_gene_names)
data_initial['Interactor B Genes'] = data_initial['Alias(es) interactor B'].apply(extract_gene_names)

# Seleccionar solo las columnas de interés
columnas_interes = ['Interactor A Genes', 'Interactor B Genes', 'Host organism(s)', 'Confidence value(s)', 'Taxid interactor A', 'Taxid interactor B']
data_initial = data_initial[columnas_interes]

# Limpiar la columna 'Confidence value(s)' para extraer solo el valor numérico
data_initial['Confidence value(s)'] = data_initial['Confidence value(s)'].str.extract(r'(\d+\.\d+)', expand=False)

# Limpiar las columnas 'Taxid interactor A' y 'Taxid interactor B' para extraer el contenido del último paréntesis
def extract_last_taxid(text):
    # Extraer el contenido después de 'taxid:652616' si está presente
    if "taxid:652616" in text:
        # Buscar la secuencia taxid:652616 y extraer el contenido entre los paréntesis
        match = re.search(r'taxid:652616\("([^"]+)"\)', text)
        if match:
            return match.group(1)
    
    # Si no se encuentra taxid:652616, realizar la limpieza habitual
    # Primera limpieza para eliminar comillas y paréntesis incompletos
    cleaned_text = re.sub(r'\s*".*$', '', text)
    # Encontrar el último paréntesis válido
    matches = re.findall(r'taxid:[\d-]+\(([^)]+)\)', cleaned_text)
    return matches[-1] if matches else text

data_initial['Taxid interactor A'] = data_initial['Taxid interactor A'].apply(extract_last_taxid)
data_initial['Taxid interactor B'] = data_initial['Taxid interactor B'].apply(extract_last_taxid)
# Limpiar la columna 'Host organism(s)' para extraer solo el contenido del último paréntesis
data_initial['Host organism(s)'] = data_initial['Host organism(s)'].apply(extract_last_taxid)
# Filtrar los valores de Confidence value(s) menores a 0.9
data_initial = data_initial[data_initial['Confidence value(s)'].astype(float) >= 0.9]



# Cargar el archivo CSV con los datos filtrados y con Fold Change
file_path_fc = 'deSeqRes_filtrado_Fusobacterium_Nucleatum_with_FC.csv'
data_with_fc = pd.read_csv(file_path_fc)

# Renombrar las columnas
data_with_fc = data_with_fc.rename(columns={
    'GeneSymbol': 'Genes',
    'DirectionalFoldChange': 'FoldChange'
})

# Crear una nueva columna 'Regulation' basada en el signo de 'FoldChange'
data_with_fc['Regulation'] = data_with_fc['FoldChange'].apply(lambda x: 'UpRegulated' if x > 0 else 'DownRegulated')

# Tomar el valor absoluto de 'FoldChange'
data_with_fc['FoldChange'] = data_with_fc['FoldChange'].abs()

# Realizar el join en ambas columnas 'Interactor A Genes' y 'Interactor B Genes'
merged_data_A = data_initial.merge(data_with_fc, left_on='Interactor A Genes', right_on='Genes', how='left')
merged_data_B = data_initial.merge(data_with_fc, left_on='Interactor B Genes', right_on='Genes', how='left')

# Concatenar los resultados y eliminar duplicados
merged_data = pd.concat([merged_data_A, merged_data_B]).drop_duplicates()

# Eliminar los registros donde 'Genes' sea NaN
merged_data = merged_data.dropna(subset=['Genes'])


# Ordenar el DataFrame por 'FoldChange' de mayor a menor
merged_data = merged_data.sort_values(by='FoldChange', ascending=False)

merged_data = merged_data.drop_duplicates()

# Guardar el resultado en un nuevo archivo CSV
merged_data.to_csv('Intact_data_Fusobacterium_nucleatum.csv', index=False)

# Guardar el resultado en un nuevo archivo Excel
merged_data.to_excel('Intact_data_Fusobacterium_nucleatum.xlsx', index=False)